# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB



**Вывод**

В столбцах days_employed и total_income меньше значений, чем в остальных. Вероятно, там имеются пропуски

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
print(data['children'].value_counts())
print(data['days_employed'].value_counts(dropna=False))
print(data['dob_years'].value_counts())
print(data['education'].value_counts())
print(data['education_id'].value_counts())
print(data['family_status'].value_counts())
print(data['family_status_id'].value_counts())
print(data['gender'].value_counts())
print(data['income_type'].value_counts())
print(data['debt'].value_counts())
print(data['total_income'].value_counts(dropna=False))
print(data['purpose'].value_counts())

data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
data['total_income'] = data['total_income'].fillna(data['total_income'].mean())
data.info()





 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
 NaN            2174
-1645.463049       1
-6620.396473       1
-1238.560080       1
-3047.519891       1
                ... 
-2849.351119       1
-5619.328204       1
-448.829898        1
-1687.038672       1
-206.107342        1
Name: days_employed, Length: 19352, dtype: int64
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: d

**Вывод**

Обнаружены пропуски в двух столбцах: days_employed и total_income.Количество пропусков одинаковое в обоих столбцах: 2174, что составляет 10% от общего объёма данных в этих столбцах. Это значительная доля, поэтому пропущенные значения необходимо заменить.
В столбце days_employed есть явно выделающиеся отрицательные значения-артефакты. Поэтому чтобы избежать искажения данных заменим пропуски медианным значением с помощью метода median().
В столбце total_income распределение значений кажется более равномерным, поэтому здесь пропуски можно заменить на среднее значение методом mean().


### Замена типа данных

In [3]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,-152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,-6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


**Вывод**

Тип данных в столбцах days_employed и total_income заменили на целочисленный с помощью метода astype(), т.к. именно он позволяет позволяет переводить данные в нужный тип. 

### Обработка дубликатов

In [4]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['purpose'] = data['purpose'].str.lower()
print(data.duplicated().sum())
data = data.drop_duplicates()
print(data.duplicated().sum())


71
0


**Вывод**

Для корректного поиска дубликатов все строковые значения были приведены в нижнему регистру методом str.lower().
Затем методами duplicated() и sum() было посчитано суммарное количество дубликатов - 71, что составляет 0.3% от общего объёма данных.
Основные причины возникновения дубликатов — повторные представления, неправильное соединение данных из разных источников, ошибки пользователя при занесении информации.

### Лемматизация

In [5]:
data['purpose'] = data['purpose'].apply(m.lemmatize)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,"[покупка, , жилье, \n]"
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,"[приобретение, , автомобиль, \n]"
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,"[покупка, , жилье, \n]"
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,"[сыграть, , свадьба, \n]"


**Вывод**

С помощью функции лемматизации библиотеки pymystem3 проведена лемматизация значений в столбце purpose, методом apply() функция применена к столбцу, значения которого перезаписаны.

### Категоризация данных

In [6]:
#data.total_income.sort_values()

def total_income_level(total_income):
    
    if total_income <= 40000:
        return 'низкий'
    if total_income <= 100000:
        return 'средний'
    if total_income <=300000:
        return 'высокий'
    return 'очень высокий'

data['total_income_level'] = data['total_income'].apply(total_income_level)

data.total_income_level.value_counts()

высокий          15509
средний           4338
очень высокий     1482
низкий             125
Name: total_income_level, dtype: int64

**Вывод**

Проведена категоризация данных по размеру ежемесячного дохода (столбец total_income). Все клиенты поделены на 4 категории по уровню дохода: низкий, средний, высокий и очень высокий доход.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
data.loc[data['children'] == 20, 'children'] = data['children'].median()
data.loc[data['children'] == -1, 'children'] = data['children'].median()
data.loc[data['children'] == 5, 'children'] = data['children'].median()
data_log = data[['children','debt']]
data_log.groupby('children').mean().sort_values('debt',ascending=False).head(10)

,debt
children,
4.0,0.097561
2.0,0.094542
1.0,0.092346
3.0,0.081818
0.0,0.075371


**Вывод**

Для чистоты выборки выпадающие значения заменены медианой.
По результатам группировки можно сделать вывод, что зависимоcть между наличием детей и возвратом кредита в срок не значительна. Среди заёмщиков без детей возвратов в срок больше, но в целом значения для всех групп примерно равны.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
data_log_family_status = data[['family_status','debt']]
data_log_family_status.groupby('family_status').mean().sort_values('debt',ascending=False).head(10)

,debt
family_status,
не женат / не замужем,0.097509
гражданский брак,0.093471
женат / замужем,0.075452
в разводе,0.071130
вдовец / вдова,0.065693


**Вывод**

Группировка клиентов по семейному положению демонстрирует нам, что максимальная доля должников приходится на холостых клиентов (9.7%) и  тех, кто проживает в гражданском браке (9.3%). Несколько ниже доля должников среди женатых и разведённых клиентов (7.5% и 7.1% соответственно). Наименьшая доля должников среди вдов/вдовцов - 6.5%.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
data_log_income = data[['total_income_level','debt']]
data_log_income.groupby('total_income_level').mean().sort_values('debt',ascending=False).head(10)

,debt
total_income_level,
высокий,0.082597
низкий,0.080000
средний,0.079299
очень высокий,0.071525


**Вывод**

Различия по данному критерию не выглядят значительными. Интересно, что наибольшая доля должников приходится на клиентов с высоким уровнем дохода (8.2%), 8% и 7.9% соответственно приходятся на клиентов с низким и средним доходом. Наименьшая доля должников среди клиентов с очень высоким доходом - 7.1%.

- Как разные цели кредита влияют на его возврат в срок?

In [12]:
def dept_purpose(purpose):
    
    if 'свадьба' in purpose:
        return 'свадьба'
    if 'недвижимость' in purpose or 'жилье' in purpose:
        return 'недвижимость'
    if 'автомобиль' in purpose:
        return 'автомобиль'
    if 'образование' in purpose:
        return 'образование'
    return 'другое'

data['dept_purpose_general'] = data['purpose'].apply(dept_purpose)

data['dept_purpose_general'].value_counts()

data_log_purpose_debt = data[['dept_purpose_general','debt']]
data_log_purpose_debt.groupby('dept_purpose_general').mean().sort_values('debt',ascending=False).head(10)

,debt
dept_purpose_general,
автомобиль,0.093590
образование,0.092200
свадьба,0.080034
недвижимость,0.072334


**Вывод**

Цели кредита всех кредиторов объединены в 4 укрупнённые группы по категории трат: автомобиль, образование, свадьба и недвижимость. Результаты группировки по цели показывают нам, что реже всего возвращают в срок кредиты, взятые на совершение операций с автомобилем (9.3%). Почти такой же процент должников среди взявших кредит на образование (9.2%). Доля должников среди тех, кто брал кредит на свадьбу - 8% и ещё меньше - 7.2% - среди тех, кто брал кредит на операции с недвижимостью. 

## Шаг 4. Общий вывод

По результатам анализа представленных данных можно сделать вывод о том, что зависимоcть между наличием детей и возвратом кредита в срок не значительна. Среди заёмщиков без детей возвратов в срок больше, но в целом значения для всех групп примерно равны. Объяснить это можно тем, что заёмщикам без детей проще планировать свой бюджет, т.к. как правило в таких случаях количество непредвиденных трат несколько сокращается по сравнению с теми, у кого есть дети.
Группировка клиентов по семейному положению демонстрирует нам, что максимальная доля должников приходится на холостых клиентов (9.7%) и  тех, кто проживает в гражданском браке (9.3%). Несколько ниже доля должников среди женатых и разведённых клиентов (7.5% и 7.1% соответственно). Наименьшая доля должников среди вдов/вдовцов - 6.5%.
Что касается влияния уровня дохода на своевременный возврат кредита, выявлено, что наибольшая доля должников приходится на клиентов с высоким уровнем дохода (8.2%), 8% и 7.9% соответственно приходятся на клиентов с низким и средним доходом. Наименьшая доля должников среди клиентов с очень высоким доходом - 7.1%. Такое распределение может быть связано с тем, что обладатели высокого дохода не всегда расчитывают свои силы и берут слишком большие кредиты, не думая о последствиях, или же не сильно беспокоятся о начислении процентов, в то время как клиенты с низким и средним доходам относятся более аккуратно к данному вопросу: тщательнее рассчитывают размер кредита и внимательнее следят за его погашением, чтобы избежать начисления процентов.

